In [16]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from random import shuffle

In [17]:
class Dataset:
    def __init__(self):
        self.df = pd.read_csv('train.csv')
        self.df['Image_ID'] = self.df['Image_ID'].apply(lambda x: '{}.jpg'.format(x))
        self.train_datagen = tf.keras.preprocessing.image.ImageDataGenerator()
        self.test_datagen = tf.keras.preprocessing.image.ImageDataGenerator()
        self.validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator()
        train_df, val_df, test_df = self.train_validate_test_split()

        self.train_ds = self.train_datagen.flow_from_dataframe(train_df,directory = 'Images', x_col = 'Image_ID', y_col = 'class', target_size=(512, 512), class_mode="categorical")
        self.test_ds = self.test_datagen.flow_from_dataframe(val_df,directory = 'Images', x_col = 'Image_ID', y_col = 'class', target_size=(512, 512), class_mode="categorical")
        self.val_ds = self.validation_datagen.flow_from_dataframe(test_df,directory = 'Images', x_col = 'Image_ID', y_col = 'class', target_size=(512, 512), class_mode="categorical")

    def train_validate_test_split(self, train_percent=.8, validate_percent=.1, seed=None):

        train = self.df.sample(frac=train_percent, random_state=25)
        validate = train.sample(frac=validate_percent, random_state=25)
        test = self.df.drop(train.index)
        train = train.drop(validate.index)


        print(train.size)
        print(test.size)
        print(validate.size)

        return train, validate, test

In [18]:
dss = Dataset()
train_ds = dss.train_ds
test_ds = dss.test_ds
val_ds = dss.val_ds

AlreadyExistsError: Another metric with the same name already exists.

In [ ]:
for _ in range(5):
    img, label = val_ds.next()  #  (1,256,256,3)
    plt.imshow((img[0]).astype(np.uint8),interpolation=None, label=label)
    plt.show()

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(512, 512, 3)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(3))

model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:

history = model.fit(train_ds,
                    steps_per_epoch = len(train_ds),
                    validation_data = val_ds,
                    validation_steps = len(val_ds),
                    epochs=5)

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_ds, verbose=2)
